In [182]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import plotly_express as px
import plotly.graph_objects as go


In [183]:
def single_bank_type(bank_type, sa, last_days, file_name):

    banks_df = pd.read_csv(f'{file_name}').iloc[-last_days:,:]
    banks_df.set_index('Series Description', inplace=True, drop=True)
    banks_df = banks_df.astype(float)
    banks_df.index = banks_df.index.astype('datetime64[ns]')

    # ----------------------------------- ASSETS ------------------------------------

    banks_df[f'LESS: Allowance for loan and lease losses, {bank_type}, {sa}'] = banks_df[f'LESS: Allowance for loan and lease losses, {bank_type}, {sa}'] * -1
    col_asset = [
        f'Treasury and agency securities, {bank_type}, {sa}', 
        f'Other securities, {bank_type}, {sa}',
        f'Commercial and industrial loans, {bank_type}, {sa}', 
        f'Real estate loans, {bank_type}, {sa}',   
        f'Consumer loans, {bank_type}, {sa}',
        f'Other loans and leases: All other loans and leases, {bank_type}, {sa}',
        f'LESS: Allowance for loan and lease losses, {bank_type}, {sa}',
        f'Cash assets, {bank_type}, {sa}',
        f'Total fed funds sold and securities purchased under agreements to resell, {bank_type}, {sa}',
        f'Loans to commercial banks, {bank_type}, {sa}',
        f'Other assets, {bank_type}, {sa}'
    ]
    asset_df = banks_df[col_asset]
    asset_percent_df = asset_df.div(banks_df[f'Total assets, {bank_type}, {sa}'], axis=0)

    
    fig = px.bar(asset_df, template='plotly_white')
    fig.update_layout(
        title=(f'{bank_type}: Assets'.capitalize()),
        xaxis_title='Date',
        yaxis_title='Value (1000000 multiplier)',
        legend_title='Legend'
    )
    fig.show()

    fig = px.bar(asset_percent_df, template='plotly_white')
    fig.update_layout(
        title=f'{bank_type}: Assets as % of total assets'.capitalize(),
        xaxis_title='Date',
        yaxis_title='% of total assets',
        legend_title='Legend'
    )
    fig.show()

    diff_asset_1w_df = asset_df -asset_df.shift(1)
    fig = px.bar(diff_asset_1w_df,  template='plotly_white')
    fig.update_layout(
        title=f'{bank_type}: assets wow change'.capitalize(),
        xaxis_title='Date',
        yaxis_title='Value (1000000 multiplier)',
        legend_title='Legend'
    )
    fig.show()
    
    # ----------------------------------- LIABILITIES ------------------------------------

    liabilities_col = [
        f'Deposits, {bank_type}, {sa}',
        f'Borrowings, {bank_type}, {sa}',
        f'Net due to related foreign offices, {bank_type}, {sa}',
        f'Other liabilities, {bank_type}, {sa}'
    ]
    liabilities_df = banks_df[liabilities_col]

    liabilities_percent = liabilities_df.div(banks_df[f'Total liabilities, {bank_type}, {sa}'], axis=0)

    fig = px.bar(liabilities_df, template='plotly_white')
    fig.update_layout(
        title=f'{bank_type}: Liabilities'.capitalize(),
        xaxis_title='Date',
        yaxis_title='Value (1000000 multiplier)',
        legend_title='Legend'
    )
    fig.show()

    fig = px.bar(liabilities_percent, template='plotly_white')
    fig.update_layout(
        title=f'{bank_type}: Liabilities as % of total liabilities'.capitalize(),
        xaxis_title='Date',
        yaxis_title='% value ',
        legend_title='Legend'
    )
    fig.show()

    diff_liabilities_1w_df = liabilities_df -liabilities_df.shift(1)
    fig = px.bar(diff_liabilities_1w_df, title='Value change 1w', template='plotly_white')
    fig.update_layout(
        title=f'{bank_type}: liabilities wow change'.capitalize(),
        xaxis_title='Date',
        yaxis_title='Value (1000000 multiplier) ',
        legend_title='Legend'
    )
    fig.show()



# SMALL BANKS

In [195]:
single_bank_type(bank_type = 'small domestically chartered commercial banks', sa= 'not seasonally adjusted', last_days = 52, file_name = 'FRB_H8_small.csv')


In [198]:
single_bank_type(bank_type = 'large domestically chartered commercial banks', sa= 'not seasonally adjusted', last_days = 52, file_name = 'FRB_H8_large.csv')

In [185]:
# WoW CHANGE GRAPH

def banks_bs_comp_change(reports, bank_type, sa, comp_criterias, last_x_days):
    banks_df = pd.DataFrame()
    for i in reports:
        a = pd.read_csv(i).iloc[-last_x_days:,:]
        a.set_index('Series Description', inplace=True, drop=True)

        banks_df = pd.concat([banks_df, a], axis=1)

    banks_df = banks_df.astype(float)
    banks_df.index = banks_df.index.astype('datetime64[ns]')

    # correct for the negative fields
    for element in bank_type: 
        banks_df[f'LESS: Allowance for loan and lease losses, {element}, {sa}'] = banks_df[f'LESS: Allowance for loan and lease losses, {element}, {sa}'] * -1

    comp_df = pd.DataFrame(index=banks_df.index)

    for i in comp_criterias:
        for z in bank_type:
            comp_df = pd.concat([comp_df, banks_df[f'{i}, {z}, {sa}']], axis=1)

    comp_df = comp_df - comp_df.shift(1)

    
    fig = px.bar(comp_df,template='plotly_white')

    fig.update_layout(
        title = 'comparison'.capitalize(),
        xaxis_title='Date',
        yaxis_title='Value (1000000 multiplier) ',
        legend_title='Legend'
    )
    fig.show()


In [186]:
# WoW CHANGE GRAPH DIVIDER
 
def banks_bs_comp_change_div(reports, bank_type, sa, comp_criterias, last_x_days, divider):
    banks_df = pd.DataFrame()
    for i in reports:
        a = pd.read_csv(i).iloc[-last_x_days:,:]
        a.set_index('Series Description', inplace=True, drop=True)

        banks_df = pd.concat([banks_df, a], axis=1)

    banks_df = banks_df.astype(float)
    banks_df.index = banks_df.index.astype('datetime64[ns]')

    # correct for the negative fields
    for element in bank_type: 
        banks_df[f'LESS: Allowance for loan and lease losses, {element}, {sa}'] = banks_df[f'LESS: Allowance for loan and lease losses, {element}, {sa}'] * -1

    comp_df = pd.DataFrame(index=banks_df.index)

    for i in comp_criterias:
        for z in bank_type:
            comp_df = pd.concat([comp_df, banks_df[f'{i}, {z}, {sa}']], axis=1)

    comp_df = comp_df - comp_df.shift(1)

    for i in comp_criterias:
        for z in bank_type:
            comp_df[f'{i}, {z}, {sa}'] = comp_df[f'{i}, {z}, {sa}']/ banks_df[f'{divider}, {z}, {sa}']

    
    fig = px.bar(comp_df,template='plotly_white')

    fig.update_layout(
        title = f"comparison, value divided by each bank's {divider}".capitalize(),
        xaxis_title='Date',
        yaxis_title='Value (1000000 multiplier) ',
        legend_title='Legend'
    )
    fig.show()


In [187]:
# VALUE AS % OF THE SUM OF THE TWO

def banks_bs_comp_percent(reports, bank_type, sa, comp_criterias, last_x_days):
    banks_df = pd.DataFrame()
    for i in reports:
        a = pd.read_csv(i).iloc[-last_x_days:,:]
        a.set_index('Series Description', inplace=True, drop=True)

        banks_df = pd.concat([banks_df, a], axis=1)

    banks_df = banks_df.astype(float)
    banks_df.index = banks_df.index.astype('datetime64[ns]')

    # correct for the negative fields
    for element in bank_type: 
        banks_df[f'LESS: Allowance for loan and lease losses, {element}, {sa}'] = banks_df[f'LESS: Allowance for loan and lease losses, {element}, {sa}'] * -1

    comp_df = pd.DataFrame(index=banks_df.index)

    for i in comp_criterias:
        for z in bank_type:
            comp_df = pd.concat([comp_df, banks_df[f'{i}, {z}, {sa}']], axis=1)

    comp_sum_df = comp_df.sum(axis=1)

    for i in range(comp_df.shape[1]):
        comp_df.iloc[:,i] = comp_df.iloc[:,i] / comp_sum_df

    fig = px.bar(comp_df,template='plotly_white')

    fig.update_layout(
        title = 'comparison'.capitalize(),
        xaxis_title='Date',
        yaxis_title='Value (1000000 multiplier) ',
        legend_title='Legend'
    )
    fig.show()


In [188]:
# ABSOLUTE VALUE GRAPH

def banks_bs_comp_absol(reports, bank_type, sa, comp_criterias, last_x_days):
    banks_df = pd.DataFrame()
    for i in reports:
        a = pd.read_csv(i).iloc[-last_x_days:,:]
        a.set_index('Series Description', inplace=True, drop=True)

        banks_df = pd.concat([banks_df, a], axis=1)

    banks_df = banks_df.astype(float)
    banks_df.index = banks_df.index.astype('datetime64[ns]')

    # correct for the negative fields
    for element in bank_type: 
        banks_df[f'LESS: Allowance for loan and lease losses, {element}, {sa}'] = banks_df[f'LESS: Allowance for loan and lease losses, {element}, {sa}'] * -1

    comp_df = pd.DataFrame(index=banks_df.index)

    for i in comp_criterias:
        for z in bank_type:
            comp_df = pd.concat([comp_df, banks_df[f'{i}, {z}, {sa}']], axis=1)

    fig = px.bar(comp_df,template='plotly_white')

    fig.update_layout(
        title = 'comparison'.capitalize(),
        xaxis_title='Date',
        yaxis_title='Value (1000000 multiplier) ',
        legend_title='Legend'
    )
    fig.show()

In [189]:
# PERCENT w DIVIDER

def banks_bs_comp_percent_div(reports, bank_type, sa, comp_criterias, last_x_days, divider):
    banks_df = pd.DataFrame()
    for i in reports:
        a = pd.read_csv(i).iloc[-last_x_days:,:]
        a.set_index('Series Description', inplace=True, drop=True)

        banks_df = pd.concat([banks_df, a], axis=1)

    banks_df = banks_df.astype(float)
    banks_df.index = banks_df.index.astype('datetime64[ns]')

    # correct for the negative fields
    for element in bank_type: 
        banks_df[f'LESS: Allowance for loan and lease losses, {element}, {sa}'] = banks_df[f'LESS: Allowance for loan and lease losses, {element}, {sa}'] * -1

    comp_df = pd.DataFrame(index=banks_df.index)

    for i in comp_criterias:
        for z in bank_type:
            banks_df[f'{i}, {z}, {sa}'] = banks_df[f'{i}, {z}, {sa}']/ banks_df[f'{divider}, {z}, {sa}']

    for i in comp_criterias:
        for z in bank_type:
            comp_df = pd.concat([comp_df, banks_df[f'{i}, {z}, {sa}']], axis=1)

    fig = px.bar(comp_df,template='plotly_white')

    fig.update_layout(
        title = f"comparison, value divided by each bank's {divider}".capitalize(),
        xaxis_title='Date',
        yaxis_title='Value (1000000 multiplier) ',
        legend_title='Legend'
    )
    fig.show()

# LIABILITIES
Comparison of liabilities: deposit dropped by borrowing ofsetted the the loss in deposit more than prioportionally 

In [197]:
banks_bs_comp_change(reports = ['FRB_H8_large.csv', 'FRB_H8_small.csv'],
bank_type = ['large domestically chartered commercial banks', 'small domestically chartered commercial banks'],
sa = 'not seasonally adjusted',
comp_criterias = ['Deposits', 'Borrowings'],
last_x_days = 100)

## DEPOSITS

In [191]:
banks_bs_comp_change_div(reports = ['FRB_H8_large.csv', 'FRB_H8_small.csv'],
bank_type = ['large domestically chartered commercial banks', 'small domestically chartered commercial banks'],
sa = 'not seasonally adjusted',
comp_criterias = ['Deposits'],
last_x_days = 100,
divider= 'Total liabilities'
)

# LOANS

In [192]:
banks_bs_comp_percent(reports = ['FRB_H8_large.csv', 'FRB_H8_small.csv'],
bank_type = ['large domestically chartered commercial banks', 'small domestically chartered commercial banks'],
sa = 'not seasonally adjusted',
comp_criterias = ['Loans and leases in bank credit'],
last_x_days = 100,)

## REAL ESTATE LOANS 
Looking at real estate we can see that small banks give out slightley more than 50% of real estate loans BUT we can see that for small banks real estate loans represent a far larger portion of their assets compared to large banks where real estate loans are a smaller percentage of assets

In [193]:
banks_bs_comp_percent(reports = ['FRB_H8_large.csv', 'FRB_H8_small.csv'],
bank_type = ['large domestically chartered commercial banks', 'small domestically chartered commercial banks'],
sa = 'not seasonally adjusted',
comp_criterias = ['Real estate loans'],
last_x_days = 100)

In [194]:
banks_bs_comp_percent_div(reports = ['FRB_H8_large.csv', 'FRB_H8_small.csv'],
bank_type = ['large domestically chartered commercial banks', 'small domestically chartered commercial banks'],
sa = 'not seasonally adjusted',
comp_criterias = ['Real estate loans'],
last_x_days = 100,
divider= 'Total assets')